In [2]:
import re
import pandas as pd
import numpy as np
import pickle

credits_url = "./data/credits.csv"
keywords_url = "./data/keywords.csv"
links_url = "./data/links.csv"
links_small_url = "./data/links_small.csv"
metadata_url = "./data/movies_metadata.csv"
ratings_url = "./data/ratings_small.csv"

In [3]:
def make_list_from_str(string):
    if isinstance(string, str):
        result_list = eval(string)
    else:
        result_list = np.nan
    return result_list

In [4]:
def make_collecton_from_str(string):
    if isinstance(string, str):
        result_dict = eval(string)
        if type(result_dict) == dict:
            result = result_dict['name']
            return result
    result = "no collection"
    return result

In [5]:
def make_bool_from_str(string):
    if string == "True":
        result = True
    elif string == "False":
        result = False
    else:
        result = np.nan
    return result

# credits.csv、整形データ

データ数：　45476

- cast: キャストのデータ（dict型）のリスト
- crew: 役職者について
- id: movie_metadataのidに対応

In [6]:
credits_df = pd.read_csv(credits_url)

print(type(credits_df.cast[0]), "　　　　castについて、str型をlist型に・・・")
credits_df.cast = credits_df.cast.map(make_list_from_str)
print(type(credits_df.cast[0]), "　　　　cast完了\n")

print(type(credits_df.crew[0]), "　　　　crewについて、str型をlist型に・・・")
credits_df.crew = credits_df.crew.map(make_list_from_str)
print(type(credits_df.crew[0]), "　　　　crew完了\n")

credits_df = credits_df.dropna()
credits_df.info()

<class 'str'> 　　　　castについて、str型をlist型に・・・
<class 'list'> 　　　　cast完了

<class 'str'> 　　　　crewについて、str型をlist型に・・・
<class 'list'> 　　　　crew完了

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [7]:
with open('credits.pickle', 'wb') as f:
    pickle.dump(credits_df, f)

# keywords.csv、整形データ

データ数：　46419

- id: movie_metadataのidに対応
- keywords: 映画を説明するキーワードのリスト

In [8]:
keywords_df = pd.read_csv(keywords_url)
keywords_df.keywords = keywords_df.keywords.map(make_list_from_str)
keywords_df = keywords_df.dropna()
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46419 entries, 0 to 46418
Data columns (total 2 columns):
id          46419 non-null int64
keywords    46419 non-null object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [10]:
with open('keywords.pickle', 'wb') as f:
    pickle.dump(keywords_df, f)

# links.csv、整形データ

データ数：　45624

- movieId: 行番号に対応するid
- imdbId: tt0を前につけると、movie_metadataのimdb_idに対応するid
- tmbdId: これが、movie_metadataのidに対応する

In [11]:
links_df = pd.read_csv(links_url)
links_df = links_df.dropna()
links_df[['tmdbId']] = links_df[['tmdbId']].astype('int')
links_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45624 entries, 0 to 45842
Data columns (total 3 columns):
movieId    45624 non-null int64
imdbId     45624 non-null int64
tmdbId     45624 non-null int64
dtypes: int64(3)
memory usage: 1.4 MB


In [12]:
with open('links.pickle', 'wb') as f:
    pickle.dump(links_df, f)

# movie_metadata.csv、整形データ

今回抽出する列

- budget: 制作費（USD） int型
- genres: 映画ごとにジャンル（dict型）のリスト
- id: TMDBのidに対応しているっぽい
- original_language: 公開時の言語
- original_title: 公開時のタイトル
- release_date: 公開日
- revenue: 興行収入（USD） float型
- spoken_languages: 話されている言語(dict型)
- title: タイトル（原題ではなく、英語版のタイトル）
- vote_average: 0.0-10.0 で、レーティングの平均
- vote_count: 投票総数
- collection: どのシリーズものに属するか（シリーズものでない場合、"no collection"）

もしかしたら今後使う列
 - belong_to_collection : "name"だけ抽出して、"colleciton"として抽出
 - homepage: ホームページのURL、使うとするならgoogle analyticsで人気度調べたり？
 - overview: ストーリー（長文。自然言語処理が必要）
 - runtime: 上映時間（分）
 - popularity: イマイチなんの指標かわからないから削除（レーティングとかをもとに算出？）
 - tagline: 短い説明。かぶりもあるけど、自然言語処理が必要

In [3]:
metadata_df = pd.read_csv(metadata_url)

/Users/labuser/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
metadata_df = metadata_df[["belongs_to_collection", "budget", "genres", "id", "original_language", "original_title", "release_date", "revenue", "spoken_languages", "title", "vote_average", "vote_count"]]
metadata_df.head()

,belongs_to_collection,budget,genres,id,original_language,original_title,release_date,revenue,spoken_languages,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,1995-10-30,373554033.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,1995-12-15,262797249.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,1995-12-22,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,1995-12-22,81452156.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,1995-02-10,76578911.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0


In [15]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 12 columns):
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
id                       45466 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
spoken_languages         45460 non-null object
title                    45460 non-null object
vote_average             45460 non-null float64
vote_count               45460 non-null float64
dtypes: float64(3), object(9)
memory usage: 4.2+ MB


In [16]:
metadata_df["collection"] = metadata_df.belongs_to_collection.map(make_collecton_from_str)
metadata_df = metadata_df.drop("belongs_to_collection", axis=1)

In [17]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 12 columns):
budget               45466 non-null object
genres               45466 non-null object
id                   45466 non-null object
original_language    45455 non-null object
original_title       45466 non-null object
release_date         45379 non-null object
revenue              45460 non-null float64
spoken_languages     45460 non-null object
title                45460 non-null object
vote_average         45460 non-null float64
vote_count           45460 non-null float64
collection           45466 non-null object
dtypes: float64(3), object(9)
memory usage: 4.2+ MB


In [28]:
number = 0
for i in range(45466):
    try:
        int(metadata_df["budget"][i])
    except Exception:
        print(metadata_df["budget"][i])
        print(i)
        number += 1
print(number)

/ff9qCepilowshEtG2GYWwzt2bs4.jpg
19730
/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
29503
/zaSf5OG7V8X8gqFvly88zDdRm46.jpg
35587
3


In [40]:
metadata_df = metadata_df.drop([19730, 29503, 35587])

In [41]:
metadata_df["budget"] = metadata_df["budget"].map(int)
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 12 columns):
budget               45463 non-null int64
genres               45463 non-null object
id                   45463 non-null object
original_language    45452 non-null object
original_title       45463 non-null object
release_date         45376 non-null object
revenue              45460 non-null float64
spoken_languages     45460 non-null object
title                45460 non-null object
vote_average         45460 non-null float64
vote_count           45460 non-null float64
collection           45463 non-null object
dtypes: float64(3), int64(1), object(8)
memory usage: 4.5+ MB


In [44]:
metadata_df.genres = metadata_df.genres.map(make_list_from_str)
metadata_df.spoken_languages = metadata_df.spoken_languages.map(make_list_from_str)

In [48]:
metadata_df.id = metadata_df.id.map(int)
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 12 columns):
budget               45463 non-null int64
genres               45463 non-null object
id                   45463 non-null int64
original_language    45452 non-null object
original_title       45463 non-null object
release_date         45376 non-null object
revenue              45460 non-null float64
spoken_languages     45460 non-null object
title                45460 non-null object
vote_average         45460 non-null float64
vote_count           45460 non-null float64
collection           45463 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 4.5+ MB


In [50]:
metadata_df = metadata_df.dropna()
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45365 entries, 0 to 45465
Data columns (total 12 columns):
budget               45365 non-null int64
genres               45365 non-null object
id                   45365 non-null int64
original_language    45365 non-null object
original_title       45365 non-null object
release_date         45365 non-null object
revenue              45365 non-null float64
spoken_languages     45365 non-null object
title                45365 non-null object
vote_average         45365 non-null float64
vote_count           45365 non-null float64
collection           45365 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 4.5+ MB


In [51]:
with open('movie_metadata.pickle', 'wb') as f:
    pickle.dump(metadata_df, f)

In [25]:
metadata_df[metadata_df.vote_average > 8][["original_title", "vote_average", "vote_count"]].head(50)

,original_title,vote_average,vote_count
46,Se7en,8.1,5915.0
49,The Usual Suspects,8.1,3334.0
109,Taxi Driver,8.1,2632.0
186,Reckless,10.0,1.0
256,Star Wars,8.1,6778.0
289,Léon,8.2,4293.0
292,Pulp Fiction,8.3,8670.0
314,The Shawshank Redemption,8.5,8358.0
351,Forrest Gump,8.2,8147.0
394,Girl in the Cadillac,10.0,1.0


In [22]:
metadata_df.groupby("vote_average", as_index = False)["original_title"].count()

,vote_average,original_title
0,0.0,2998
1,0.5,13
2,0.7,1
3,1.0,105
4,1.1,1
5,1.2,4
6,1.3,13
7,1.4,5
8,1.5,30
9,1.6,6


In [146]:
metadata_df[["collection", "id"]].dropna().groupby("collection", as_index=False)["id"].count()

,collection,id
0,... Has Fallen Collection,2
1,00 Schneider Filmreihe,1
2,08/15 Collection,1
3,100 Girls Collection,2
4,101 Dalmatians (Animated) Collection,2
5,101 Dalmatians (Live-Action) Collection,2
6,12 Rounds Collection,3
7,1920 Collection,2
8,2 Days In... Collection,2
9,2001 Maniacs Collection,1


In [ ]:
test_list2 = eval(keywords_df.keywords[0])
print(test_list2)
print(type(test_list2[0]))

In [ ]:
code = "keywords_df.keywords[0] = "
exec(code + keywords_df.keywords[0])
for keys in list_of_keys:
    print(type(keys))
    print(keys)

In [ ]:
print(type(keywords_df.keywords[2]))

# ratings_small.csvについて

In [77]:
rating_df = pd.read_csv(ratings_url)
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [78]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [82]:
rating_df.groupby(['userId'], as_index=False).head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
20,2,10,4.0,835355493
21,2,17,5.0,835355681
22,2,39,5.0,835355604
23,2,47,4.0,835355552
24,2,50,4.0,835355586
